In [ ]:
"""
Created on Thu Feb 16 19:54 2023

Use Jeremie's basins to define a mask and then fine-tune for a few imprecisions
=> this mask can then be used to define the ice shelves in the ice-shelf mask routines

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import matplotlib as mpl
from tqdm.notebook import tqdm
import basal_melt_param.plume_functions as pf
from scipy.interpolate import griddata

In [ ]:
inputpath_raw = '/bettik/burgardc/DATA/NN_PARAM/raw/'

In [ ]:
# needed for the old Nisf numbers and defining the domain
inputpath_data='/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_OPM016/'
file_isf = xr.open_dataset(inputpath_data + 'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')



READ THE BASINS AND EXTRAPOLATE A BIT OUT INTO THE OCEAN TO BE SURE TO CATCH THE EDGES

In [ ]:
drainage_basins_file = xr.open_dataset(inputpath_raw + 'IceBoundaries_Y2015-2017_Antarctica_v2.2.1.nc')
drainage_basins = drainage_basins_file['Ice_Shelf_Basins_ID'].sel(x=file_isf.x,y=file_isf.y)

Iceshelf = drainage_basins
# extrapolation of nan data
X2d, Y2d = np.meshgrid(Iceshelf['x'], Iceshelf['y'])
mxy = Iceshelf['x'].size * Iceshelf['y'].size
mask_variable1d = np.reshape(Iceshelf.values, mxy)
X1d = np.reshape(X2d, mxy)
Y1d = np.reshape(Y2d, mxy)
mask_variable1d_nonan = mask_variable1d[ ~np.isnan(mask_variable1d)]
X1d_nonan = X1d[ ~np.isnan(mask_variable1d)]
Y1d_nonan = Y1d[ ~np.isnan(mask_variable1d)]
tx1d = griddata((X1d_nonan,Y1d_nonan), mask_variable1d_nonan, (X1d,Y1d), method='nearest', fill_value=np.nan)
mask_variable2d = np.reshape(tx1d,(Iceshelf['y'].size, Iceshelf['x'].size))
Iceshelf['ID_isf'] = (('y', 'x'), mask_variable2d)


isf_names_file = xr.open_dataset(inputpath_raw + 'Melt_Rignot_Adusumilli_Basins.nc')
summary_mask_basins = xr.merge([isf_names_file['name_isf'],isf_names_file['name_reg'],Iceshelf['ID_isf'].rename({'ID_isf': 'Nisf'})])

missing_names = xr.DataArray(data=np.array(['East Ant1 small isf','Larsen A']), dims=['Nisf'])
missing_names = missing_names.assign_coords({'Nisf': [12, 96]}).rename('name_isf')
missing_regs = xr.DataArray(data=np.array(['East 1','Weddell']), dims=['Nisf'])
missing_regs = missing_regs.assign_coords({'Nisf': [12, 96]}).rename('name_reg')

summary_mask_basins = xr.merge([summary_mask_basins,missing_names,missing_regs])

coloc_list = []
for ID_basin in summary_mask_basins['Nisf'].values:
    for ID_isf in file_isf['Nisf'].values:
        if summary_mask_basins['name_isf'].sel(Nisf=ID_basin).values == file_isf['isf_name'].sel(Nisf=ID_isf):
            coloc_list.append(xr.DataArray(data=np.array(ID_isf)).rename('Nisf_orig').assign_coords({'Nisf': ID_basin}))
            
summary_mask_basins['Nisf_orig'] = xr.concat(coloc_list, dim='Nisf')
summary_mask_basins.to_netcdf('/bettik/burgardc/DATA/NN_PARAM/interim/basins_mask_extrap_50km.nc')

In [ ]:
summary_mask_basins

MERGING RELEVANT SHELVES

In [ ]:
summary_mask_basins = xr.open_dataset('/bettik/burgardc/DATA/NN_PARAM/interim/basins_mask_extrap_50km.nc')

In [ ]:
file_msk = file_mask_cut['ls_mask012']
isf_only_mask = file_msk.where((file_msk > 0) & (file_msk < 2))
isf_mask_all = summary_mask_basins['ID_isf'].where(np.isfinite(isf_only_mask))

In [ ]:
isf_mask_withRossmerged = isf_mask_all.where(isf_mask_all != 58, 57)
isf_mask_withRossFRISmerged = isf_mask_withRossmerged.where(isf_mask_withRossmerged != 104, 103)
isf_mask_withRossFRISStangemerged = isf_mask_tt_withRossFRISmerged.where(isf_mask_withRossFRISmerged != 82, 83)
isf_mask_merged1 = isf_mask_withRossFRISStangemerged.where(isf_mask_withRossFRISStangemerged != 157, 89)
isf_mask_merged2 = isf_mask_merged1.where(isf_mask_merged1 != 156, 89)
isf_mask_merged1 = isf_mask_merged2.where(isf_mask_merged2 != 149, 89)
isf_mask_merged2 = isf_mask_merged1.where(isf_mask_merged1 != 147, 89)
isf_mask_merged1 = isf_mask_merged2.where(isf_mask_merged2 != 148, 84)
isf_mask_merged2 = isf_mask_merged1.where(isf_mask_merged1 != 127, 84)
isf_mask_merged1 = isf_mask_merged2.where(isf_mask_merged2 != 146, 85)
isf_mask_merged1 = isf_mask_merged2.where(isf_mask_merged2 != 153, 72)